# Topic Modelling on Program Source Code
---
By Kishalay Banerjee, Dan Jones and Sam Harding

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 0y

In [3]:
import pandas
import pickle
import numpy
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from urllib.request import urlretrieve

In [5]:
numpy.random.seed(0xD00D5) 

## Preparation

There are a number of files which are too large to store on GitHub. These are hosted on our server, and can be downloaded by running the following cell:

In [ ]:
files = [
    'full_lda_model.pickle',
    'full_tf.pickle',
    'full_tf_vectorizer.pickle',
    'full-dataset.csv.gz',
]

base_url = 'https://daniel.wilshirejones.com/private-uUX6IzfsRYLNiti4ZFmgv6U3dFInnq37r5YSQs46iejeB96q0MAy9Ko7hkgo/'
destination_directory = '../data/'

for file in files:
    url = base_url + file
    destination = destination_directory + file
    print("Downloading '{}'' to location '{}'".format(url, destination))
    urlretrieve(url, destination)

## Generating the Dataset

TODO: Import dataset.py and explain how it's used + what it does.

TODO: Add Sam's scrub function in here?

In [6]:
minimal_dataset = pandas.read_csv("../data/dataset.csv.gz", header=None, names=['repo', 'language', 'documents'])
minimal_dataset.head()

,repo,language,documents
0,28457823,javascript,"b""module.exports = {\n plugins: [\n requir..."
1,28457823,javascript,"b""// The path where to mount the REST API app\..."
2,28457823,javascript,"b""import { Observable } from 'rx';\nimport deb..."
3,28457823,javascript,"b""import { Observable } from 'rx';\n// import ..."
4,28457823,javascript,"b""import { Observable } from 'rx';\n\nmodule.e..."


In [ ]:
full_dataset = pandas.read_csv("../data/full-dataset.csv.gz", header=None, names=['repo', 'language',  'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
full_dataset = full_dataset.drop(columns='topics')

full_dataset.head()

## Topic Modelling on Individual Source Files

Basically done, just need to copy it over. Maybe run on the bigger dataset?

TODO:
  - Copy work from documentation/daniel-jones.ipynb
  - Add visualisation with pyldavis

For our purposes, common words are important and rare words aren't. So we shouldn't use tf-idf as a metric, bag-of-words makes more sense. (TODO: Maybe: "Similarly, filter out words that don't occur very often").


In [7]:
documents = minimal_dataset['documents']

In [9]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)

with open('../data/minimal_lda_tf.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/minimal_lda_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

with open('../data/minimal_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [11]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/minimal_lda_tf.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/minimal_lda_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

Try to do this on the full dataset:

In [ ]:
number_of_languages = 4
all_documents = full_dataset['documents']

full_tf_vectorizer = CountVectorizer(stop_words=None)
full_tf = full_tf_vectorizer.fit_transform(all_documents)
full_tf_feature_names = full_tf_vectorizer.get_feature_names()

full_lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
full_model = full_lda.fit(full_tf)

with open('../data/full_lda_model.pickle', 'wb') as f:
    pickle.dump(full_model, f)

with open('../data/full_tf.pickle', 'wb') as f:
    pickle.dump(full_tf, f)
    
with open('../data/full_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(full_tf_vectorizer, f)

In [ ]:
with open('../data/full_lda_model.pickle', 'rb') as f:
    full_model = pickle.load(f)

with open('../data/full_tf.pickle', 'rb') as f:
    full_tf = pickle.load(f)
    
with open('../data/full_tf_vectorizer.pickle', 'rb') as f:
    full_tf_vectorizer = pickle.load(f)

Visualize our new model. Note that the following code cell requires more than 8 GB of RAM to run.

In [ ]:
pyLDAvis.sklearn.prepare(full_model, full_tf, full_tf_vectorizer)

Reference: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

## Topic Modelling on Programming Language Mixtures
Keypoint: topics are programming languages, file with mixture of programming languages, identify which is which.

Applicability to cyber-security: identifying malware embedded within normal programs (shellcode).

### Combining Source Files per Repository

Currently our data set consists of one data point per file containing:
  1. ID of the Github repository the file belongs to.
  2. Programming language it is written in (identified by file extension).
  3. File contents
  
In this section, we extend our analysis from working on documents with one language per file, to a system where there is a mixture of languages inside each document. To do this, we combine all the files in each repository into a single data point.

In [49]:
def concat_texts(group):
    [repo_id] = group['repo'].unique()
    combined = ' '.join(group['documents'])
    return combined

In [50]:
combined_documents = minimal_dataset.groupby(by='repo').apply(concat_texts)

In [51]:
combined_documents.head()

repo
19438      b"#' @include ggplot-global.R\n#' @include ggp...
26554      b'# The contents of this file are subject to t...
544208     b'import logging\nimport os\nimport platform a...
643909     b'#\' Environment variables to set when callin...
2594513    b'# S3 method to deal with chunks and inline t...
dtype: object

### LDA Model 

In [60]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(combined_documents)

with open('../data/concatDocs_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)
    
with open('../data/concatDocs_tf.pickle', 'wb') as f:
    pickle.dump(tf, f)

In [61]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

with open('../data/concatDocs_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [52]:
with open('../data/concatDocs_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/concatDocs_tf.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/concatDocs_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [62]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.150973 -0.000614       1        1  49.041760
3     -0.016625 -0.167877       2        1  24.723611
1     -0.302164  0.065904       3        1  13.224716
0      0.167816  0.102586       4        1  13.009913, topic_info=       Category          Freq           Term         Total  loglift  logprob
term                                                                        
224987  Default  74181.000000           self  74181.000000  30.0000  30.0000
209790  Default  35155.000000          react  35155.000000  29.0000  29.0000
6203    Default  23585.000000             12  23585.000000  28.0000  28.0000
10108   Default  17361.000000             16  17361.000000  27.0000  27.0000
118970  Default  39038.000000           from  39038.000000  26.0000  26.0000
16221   Default  13050.000000             24  13050.000000  25.0000  25.0000
88624   Default  14728.000000  createsvgicon  14728.000000  24.0000  24.0000
4646    Default  18154.000000             10  18154.000000  23.0000  23.0000
20365   Default  11576.000000             32  11576.000000  22.0000  22.0000
25767   Default  11270.000000             48  11270.000000  21.0000  21.0000
256596  Default  70622.000000            the  70622.000000  20.0000  20.0000
178605  Default  15211.000000        nexport  15211.000000  19.0000  19.0000
8196    Default  11369.000000             14  11369.000000  18.0000  18.0000
93741   Default  26488.000000        default  26488.000000  17.0000  17.0000
5507    Default  12663.000000             11  12663.000000  16.0000  16.0000
15259   Default   9799.000000             21   9799.000000  15.0000  15.0000
7684    Default  10783.000000             13  10783.000000  14.0000  14.0000
182715  Default  21424.000000           none  21424.000000  13.0000  13.0000
10837   Default   9429.000000             17   9429.000000  12.0000  12.0000
118218  Default  10829.000000       fragment  10829.000000  11.0000  11.0000
238426  Default   8399.000000            str   8399.000000  10.0000  10.0000
180274  Default  20970.000000        nimport  20970.000000   9.0000   9.0000
139409  Default  33744.000000             in  33744.000000   8.0000   8.0000
17250   Default   6988.000000             26   6988.000000   7.0000   7.0000
27860   Default   5677.000000            512   5677.000000   6.0000   6.0000
9620    Default   9610.000000             15   9610.000000   5.0000   5.0000
138187  Default  74472.000000             if  74472.000000   4.0000   4.0000
210709  Default   5366.000000          realm   5366.000000   3.0000   3.0000
17808   Default   7050.000000             28   7050.000000   2.0000   2.0000
166131  Default  11499.000000        message  11499.000000   1.0000   1.0000
...         ...           ...            ...           ...      ...      ...
211325   Topic4   1043.602861      recipient   1224.776579   1.8794  -6.6986
224987   Topic4  24409.028588           self  74181.442875   0.9279  -3.5463
216694   Topic4   4817.857855         result   9477.672065   1.3628  -5.1689
166131   Topic4   5456.608823        message  11499.138735   1.2940  -5.0444
96945    Topic4   2718.527332           dict   4715.041344   1.4888  -5.7412
59130    Topic4   3665.434449    assertequal   7703.166344   1.2968  -5.4423
182715   Topic4   7077.002441           none  21424.371679   0.9318  -4.7844
166471   Topic4   1637.375695       messages   2481.306967   1.6238  -6.2482
83034    Topic4   3051.074424            com   7003.838822   1.2085  -5.6258
137816   Topic4   4631.711734             id  16570.732970   0.7647  -5.2083
56394    Topic4   2626.381485            any   6292.078536   1.1658  -5.7757
93727    Topic4   4786.163104            def  19234.092839   0.6485  -5.1755
256596   Topic4   9544.348309            the  70622.704887   0.0381  -4.4853
179191   Topic4   4171.843620          nfrom  15317.673429   0.7388  -5.3129
138187   Topic

## Identifying Program Subjects and Themes

Hypothesis: Using tf-idf rather than bag-of-words as an input to LDA will prioritise rare words. In the case of source code, this means programming language keywords (an identifying feature of programming languages) are deprioritised, and so a more human idea of topics may emerge. 

We can use repo-list.json and the repo-ids to map the github topics/tags to each repo. Might be a small/easy task to compare against the programming langauge identification.

In [ ]:
documents = minimal_dataset['documents']

In [ ]:
tf_vectorizer = TfidfVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

with open('../data/tfidf_lda_tf.pickle', 'wb') as f:
    pickle.dump(tf, model)
    
with open('../data/tfidf_lda_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, model)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

with open('../data/tfidf_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/tfidf_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/tfidf_lda_tf.pickle', 'wb') as f:
    tf = pickle.load(model)
    
with open('../data/tfidf_lda_tf_vectorizer.pickle', 'wb') as f:
    tf_vectorizer = pickle.load(model)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

This still prioritises programming language keywords. One approach to solving this problem is to consider all keywords as "stopwords". First, gather a list of R, Python and Javascript keywords:

In [ ]:
import keyword

python_keywords = keyword.kwlist
python_keywords

R reserved words (sourced from the manual: https://stat.ethz.ch/R-manual/R-devel/library/base/html/Reserved.html)

In [ ]:
r_keywords = [
    "if", 
    "else", 
    "repeat",
    "while",
    "function", 
    "for",
    "in",
    "next",
    "break",
    "TRUE",
    "FALSE",
    "NULL", 
    "Inf", 
    "NaN",
    "NA",
    "NA_integer_",
    "NA_real_",
    "NA_complex_",
    "NA_character_", 
]

Javascript keywords and reserved words (source: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Lexical_grammar#Keywords)

In [ ]:
javascript_keywords = [  # jaccard("ideal javascript topic", topic_i)
    "break",
    "case",
    "catch",
    "class",
    "const",
    "continue",
    "debugger",
    "default",
    "delete",
    "do",
    "else",
    "export",
    "extends",
    "finally",
    "for",
    "function",
    "if",
    "import",
    "in",
    "instanceof",
    "new",
    "return",
    "super",
    "switch",
    "this",
    "throw",
    "try",
    "typeof",
    "var",
    "void",
    "while",
    "with",
    "yield",
]

In [ ]:
documents = minimal_dataset['documents']

In [ ]:
tf_vectorizer = TfidfVectorizer(stop_words=javascript_keywords+python_keywords+r_keywords)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
    
with open('../data/tfidf_lda_tf_ignore_keywords.pickle', 'rb') as f:
    pickle.dump(tf, f)
    
with open('../data/tfidf_lda_tf_vectoriser_ignore_keywords.pickle', 'rb') as f:
    pickle.dump(tf_vectoriser, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/tfidf_lda_tf_ignore_keywords.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/tfidf_lda_tf_vectoriser_ignore_keywords.pickle', 'rb') as f:
    tf_vectoriser = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

## Measuring the Efficacy of Topic Models (WRITEUP: WHO?)

Main question: How do we evaluate how well a topic (from LDA for example) represents a meaningful "topic" or theme?

TODO: do some research on this??? There must be some papers etc that try to formalise this that we can borrow ideas from?

Paper dump:
  - Looks like a good summary paper: http://www.aclweb.org/anthology/E14-4005 Find more papers from this ones references?
    - " KL-divergence (Li and McCallum, 2006; Wang et al., 2009; Newman et al., 2009), cosine measure (He et al., 2009; Ramage et al., 2009) and the average Log Odds Ratio (Chaney and Blei, 2012). "
    - "Kim and Oh (2011) also applied  the  cosine  measure  and  KL-Divergence which were compared with four other measures: Jaccard’s Coefficient, Kendall’s τ coefficient, Discount  Cumulative  Gain  and  Jensen  Shannon  Divergence (JSD)."
  - Cool name haven't read it: http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf
  
We considered all of these metrics, and found th Jaccard Index to be most suitable. This was primarily due to it's use of set operations, which are invariant to ordering and number of observations.

### Reading Tea Leaves Paper
http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Word Overlap

I think this is used as a baseline measure in the summary paper above (http://www.aclweb.org/anthology/E14-4005). Should be a quick implementation so worth a try.

### Jaccard Index

From the papers above this seems to have been used relatively often for _linking machine-generated topics to human topics_ and so maybe this is a good application for it. Apparently explored here "https://link.springer.com/chapter/10.1007/978-3-642-19437-5_13" but I haven't read it.


In [ ]:
def jaccard_index(a, b):
    a = set(a)
    b = set(b)
    return len(a & b) / len(a | b)

Consider the group of language keywords as the best possible topic for each language. Compare each of our machine generated topics with each of our ideal topics by computing their Jaccard Index:

In [ ]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
len(tf_feature_names)

In [ ]:
list(enumerate(model.components_))

### Kendall’s τ Coefficient

Measures the association between two ranked lists. Source: Computational Linguistics and Intelligent Text Processing book.

### Evaluating Topic Models
TODO: better title needed

Idea:
  - save the actual % of each program langauge per repo
  - Then try to use LDA model to tell us "I believe repo <x> is 10% Topic 1, 20% Topic 2 etc". 
  - Use analysis from above two sections to create a "most likely mapping from lda topic to programming language".
  - rate our models

Here we can do cross-validation etc.

Load in the test data set:

In [27]:
test_dataset = pandas.read_csv("../data/test-dataset.csv.gz", header=None, names=['repo', 'language', 'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
test_dataset = test_dataset.drop(columns='topics')

test_dataset.head()

,repo,language,documents
0,69798748,javascript,"b""const glob = require('glob')\nconst markdown..."
1,128624453,javascript,"b""module.exports = {\n extends: ['@commitlint..."
2,128624453,javascript,"b""module.exports = {\n extends: ['standard', ..."
3,128624453,javascript,"b""const cp = require('child_process')\nconst g..."
4,128624453,javascript,"b""module.exports = {\n verbose: true,\n tran..."


To evaluate the mixture model, we must label each repository with it's percentage of each programming language:

In [41]:
def calculate_language_percentages(group):
    total_python_length = 0
    total_r_length = 0
    total_javascript_length = 0
    
    for index, repo, language, document in group.itertuples():
        if language == 'python':
            total_python_length += len(document)
            
        if language == 'javascript':
            total_javascript_length += len(document)
            
        if language == 'r':
            total_r_length += len(document)
            
    total_length = total_python_length + total_r_length + total_javascript_length
            
    return pandas.Series([
        total_python_length/total_length,
        total_r_length/total_length,
        total_javascript_length/total_length,
    ])

In [46]:
combined_test_documents = test_dataset.groupby(by='repo').apply(calculate_language_percentages)
combined_test_documents.columns = ['python', 'r', 'javascript']

Here are the programming language percentages for each of repository in our test dataset:

In [48]:
combined_test_documents

,python,r,javascript
repo,,,
596892,1.000000,0.000000,0.000000
1248263,0.652443,0.000000,0.347557
1790564,0.000000,0.000000,1.000000
4751958,0.360730,0.000000,0.639270
12465340,0.000000,0.995105,0.004895
13523710,0.000000,1.000000,0.000000
14267375,0.941701,0.000000,0.058299
14579179,0.000000,0.189083,0.810917
16146440,0.000000,0.253421,0.746579


# Notes:
  - our keyword lists have simliar/common words e.g. python javascript and r all share some keywords. this can be seen in the documents. choosing another language, with a completely different set of keywords might prove easier to differentiate for the LDA model. somethind somthing LDA uses distance, but if true topics share key words, then distance metric breaks.